In [1]:
import requests
import pandas as pd
import numpy as np
import json

from util import convert_amount, parse_currency

In [21]:
r = requests.get('http://localhost:8000/products?format=json')
shop_df = pd.DataFrame(json.loads(r.content.decode()))
shop_df['name'] = [i.lower() for i in shop_df['name']]
shop_df.head()
shop_df_ = shop_df.copy()
cols = ['name', 'price', 'currency', 'amount']
shop_df = pd.DataFrame(columns=['name', 'price', 'amount', 'unit'])
for idx, [name, price, currency, amount] in enumerate(shop_df_[cols].to_numpy()):
    amount, unit = convert_amount(amount)
    price = parse_currency(price, currency)
    shop_df.loc[idx] = [name, price, amount, unit]

In [22]:
shop_df

,name,price,amount,unit
0,sainsbury's fairtrade bananas loose,73,1000,g
1,sainsbury's carrots loose,41,1000,g
2,sainsbury's whole cucumber,43,,unit
3,sainsbury's onions loose,85,1000,g
4,sainsbury's baking potatoes loose,64,1000,g
...,...,...,...,...
1155,young's chip shop 2 extra large fish fillets i...,200,320,g
1156,viennetta biscuit caramel ice cream 650ml,165,650,ml
1157,sainsbury's strawberry gateau 610g,150,610,g
1158,sainsbury's toffee cream gateau bar 400g,200,400,g


In [23]:
filepath = 'shopping_lists/shoppingList13.json'
with open(filepath) as f:
    shopping_list = json.load(f)

In [24]:
def get_items(shopping_list):
    items_df = pd.DataFrame([i[0] for i in pd.DataFrame(shopping_list['aisles'])['items']])
    qties = pd.DataFrame(list(items_df['measures']))['metric']
    items_df.drop(['measures', 'usages', 'pantryItem', 'cost'], axis=1, inplace=True)
    items_df['amount'] = [i['amount'] for i in qties]
    items_df['unit'] = [i['unit'] for i in qties]
    items_df['name'] = items_df['name'].map(str.split)
    to_drop = ['quick', 'canned', 'cooking', 'fat', 'ground', 'leaves', 'pasta',
               'plain', 'cooked', 'full-fat', 'pieces', 'scrambled', 'frozen', 'creamy']
    items_df['name'] = [[j for j in i if j not in to_drop] for i in items_df['name']]
    return items_df

In [25]:
items_df = get_items(shopping_list)

In [26]:
items_df

,id,name,aisle,ingredientId,amount,unit
0,600627,[sprite],Beverages,14144,1.0,
1,600635,"[burrito, wrap]",Ethnic Foods,10118364,1.0,
2,600637,[tuna],Seafood,10015121,4.0,
3,600651,"[mat, beans]",Pasta and Rice,99144,1.0,
4,600655,[tomato],Produce,11529,1.0,
5,600659,[water],Pantry Items,14412,2.0,
6,600661,[chicken],Meat,5006,1.0,
7,600665,[fries],Frozen,11408,2.0,
8,600669,[bread],Bakery/Bread,18064,1.0,
9,600671,"[potato, chips]",Savory Snacks,19411,5.0,


In [29]:
def match_products(items_df: pd.DataFrame):
    matches_df = pd.DataFrame()
    for product in items_df.iloc:
        regex = ''.join([f'(?=.*{i})' for i in product['name']])   
        entries = shop_df['name'].str.contains(regex)    
        if entries.any():
            matches = shop_df[entries]
            print(f"appending {' '.join(product['name'])}")
            matches_df = matches_df.append(matches)
        else:
            print(f"no such product: {' '.join(product['name'])}")
    return matches_df

In [31]:
matches = match_products(items_df)

no such product: sprite
no such product: burrito wrap
appending tuna
appending mat beans
appending tomato
appending water
appending chicken
appending fries
appending bread
appending potato chips
appending tea
appending granola
no such product: m&m candy
appending yogurt
no such product: pizza mix


In [32]:
matches

,name,price,amount,unit
714,princes tuna chunks in spring water 4x145g (11...,325,112,g*
867,princes tuna chunks in sunflower oil 4x145g (1...,325,112,g*
711,sainsbury's baked beans in tomato sauce 200g,25,200,g
727,sainsbury's baked beans in reduced sugar & sal...,25,200,g
734,hubbard's foodstore baked beans in tomato sauc...,22,400,g
...,...,...,...,...
515,the coconut collaborative natural yogurt alter...,250,350,g
517,actimel yogurt drink strawberry & blueberry 12...,300,1200,g
558,yeo valley organic banana yogurt with nut butt...,100,130,g
560,yeo valley organic black cherry yogurt with nu...,100,130,g


In [85]:
shop_df[shop_df['name'].str.contains('(?=.*)(?=.*pasta)')]

,store,name,img_url,price,currency,amount,unit
736,sainsburys,dolmio bolognese pasta sauce original 500g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.4,GBP,500g,unit
912,sainsburys,dolmio bolognese pasta sauce original 750g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,2.0,GBP,750g,unit


### add to the database:
    - ground cumin / cumin
    - hot sauce
    - almonds, agave
    - salsa
    - tahini
    - almond milk
    - worcestershire sauce
    - flax egg
    - maple syrup
    - lemon juice
    - garlic powder
    - gluten-free
    - lime juice
    - pesto
    - mozzarella
    - cornstarch
    - green bell pepper
    - red pepper flakes
    - pumpking seeds
    - dressings (caeasar etc)
    - cashews
    - dijon mustard
    - chili sauce
    - oats
    - peanut butter
    - tortilas
    - vanilla extract
    - starch
    - vinegar (white, apple cider)
    - soda, candy